In [2]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()
f = open('../glove_data/glove.6B/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


In [6]:
import pandas as pd

In [7]:
balanced_data=pd.read_csv('balanced.csv')
balanced_data=balanced_data[['Category','Description']]
balanced_data

,Category,Description
0,Upper Body Wear Tops,Cheer on your Florida State Seminoles in this ...
1,Upper Body Wear Tops,Celebrate your Baltimore Ravens fandom during ...
2,Upper Body Wear Tops,A fau sheepskin bat sleeve Dolman top in a cre...
3,Upper Body Wear Tops,Dont let a cold snap keep you from supporting ...
4,Upper Body Wear Tops,The time has come for the Atlantic 10 Conferen...
5,Upper Body Wear Tops,Celebrate your fandom with this Miami Hurrican...
6,Upper Body Wear Tops,Cheer on your Rutgers Scarlet Knights in this ...
7,Upper Body Wear Tops,Celebrate your Florida Gators fandom with this...
8,Upper Body Wear Tops,Celebrate your Maryland Terrapins fandom with ...
9,Upper Body Wear Tops,When the season changes the nights get longer ...


In [9]:
texts=balanced_data['Description']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [11]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences)

Found 1143 unique tokens.


In [15]:
import keras
import keras.utils
from keras import utils as np_utils

In [18]:
labels =balanced_data['Category']
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (180, 128)
Shape of label tensor: (180,)


In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.33, random_state=42)

In [23]:
embeddings_index = {}
f=open('glove.6B.100d.txt')

In [24]:
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [26]:
from keras.layers import Embedding
EMBEDDING_DIM = 100
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

NameError: name 'embedding_matrix' is not defined